# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,869.709415,0.608499,1234,870.317914,869.100916,1740.635827,1738.201831
6,123.391384,-0.093310,1234,123.298074,123.484694,246.596148,246.969389
7,426.005206,0.430243,1234,426.435449,425.574963,852.870898,851.149927
8,69.997211,0.400271,1234,70.397482,69.596940,140.794964,139.193880
9,242.955475,-0.414525,1234,242.540950,243.370001,485.081900,486.740002
10,147.342725,-0.339597,1234,147.003129,147.682322,294.006258,295.364644
11,262.946310,-0.965128,1234,261.981182,263.911438,523.962364,527.822875
12,857.570585,0.791321,1234,858.361906,856.779265,1716.723811,1713.558529
13,901.430676,-0.586994,1234,900.843682,902.017670,1801.687363,1804.035340
14,737.748477,0.770011,1234,738.518488,736.978466,1477.036976,1473.956932


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-870.317914,869.100916
6,-123.298074,123.484694
7,-426.435449,425.574963
8,-70.397482,69.596940
9,-242.540950,243.370001
10,-147.003129,147.682322
11,-261.981182,263.911438
12,-858.361906,856.779265
13,-900.843682,902.017670
14,-738.518488,736.978466


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-870.317914,869.100916
6,-123.298074,123.484694
7,-426.435449,425.574963
8,-70.397482,69.596940
9,-242.540950,243.370001
10,-147.003129,147.682322
11,-261.981182,263.911438
12,-858.361906,856.779265
13,-900.843682,902.017670
14,-738.518488,736.978466


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-870.31791355,  869.10091559],
       [-123.29807398,  123.48469434],
       [-426.4354492 ,  425.57496334],
       [ -70.39748196,   69.59694022],
       [-242.54094999,  243.3700008 ],
       [-147.00312895,  147.68232196],
       [-261.98118199,  263.91143765],
       [-858.36190572,  856.77926455],
       [-900.84368169,  902.01767018],
       [-738.51848813,  736.97846609]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1738.635827,-1738.201831
6,244.596148,-246.969389
7,850.870898,-851.149927
8,138.794964,-139.193880
9,483.081900,-486.740002
10,292.006258,-295.364644
11,521.962364,-527.822875
12,1714.723811,-1713.558529
13,1799.687363,-1804.035340
14,1475.036976,-1473.956932


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
